In [4]:
using PyPlot
using Toms566

# Problem 1
Develop a Newton's method to approximate $e^{-1}$ that does not involve division. Start at $x_0 = 0.3$ and run this thing.

## Solution: 
Using $f(x) = \log(\frac{x}{d})$, we obtain the iterative method: 

$$x^{k+1} = x^k - (\log(x^k) - \log(d)) x^k$$

Now, we run it.

I don't see any significant differences between the two $x_0$ runs. One takes longer than the other?

In [5]:
function newtonRoot(x0)
    # Vars
    x = Float64[]
    d = e
    eps = 1.0e-8
    its = 0

    # First iteration
    push!(x,x0)
    push!(x, x[end] - x[end]*(log(x[end]) - log(d)))

    # Continued iterations
    while abs(x[end] - x[end-1]) > eps
        push!(x, x[end] - x[end]*(log(x[end]) - log(d)))
        its += 1
    end

    @printf "The approximation: %10f\n" x[end]
    @printf "The error: %10f\n" abs(x[end] - e)
    @printf "Iterations: %5d\n" its
end

Qt: Untested Windows version 10.0 detected!


newtonRoot (generic function with 1 method)

In [6]:
newtonRoot(0.3)

The approximation:   2.718282
The error:   0.000000
Iterations:     6


In [7]:
newtonRoot(1.0)

The approximation:   2.718282
The error:   0.000000
Iterations:     5


# Problem 2
a) Newton's method on $f(x) = x^q$. What's the convergence ratio there?

b) Apply Newton's method to minimize $f(x) = \| x \|_2^\beta$. For what $x_0$ and $\beta$ does it converge? What happens when $\beta \leq 1$?

c) Repeat part b, but with Armijo linesearch. 

## Solution
a) If we apply the straight root-finding method, then we have this iterative method:

$$x^{k+1} = x^k - \frac{f(x)}{f'(x)} = x^k - \frac{(x^k)^q}{q(x^k)^{q-1}}$$

$$= x^k - \frac{1}{q}x^k = x^k (1 - \frac{1}{q})$$

Therefore the method is Q-linear, with $\mu = 1 - \frac{1}{q}$. 

If we apply the method instead to the derivative, then through the same procedure we obtain $\mu = 1 - \frac{1}{q-1}$.


b) The method is implemented in the next code box. The following values of $\beta$ don't converge: 2,4. It's not entirely clear to me which x-values converged and which didn't.

## Regular Newton

In [78]:
function newtonMethod(obj, x0; linesearch=true, maxIts=500, optTol=1.0e-8, verbose=false)
    its = 0
    x = x0
    fvals = []
    gnormvals = []
    (f,g,H) = obj(x)
    g0 = g

    while (its < maxIts && norm(g,2) > optTol && norm(g,2) > (1.0e-4*norm(g0,2)))
        (f,g,H) = obj(x)
        
        # Modify Hessian if not positive definite
        E = eigfact(H);
        V = real(E[:vectors]);
        lambda = real(diagm(max(E[:values],1e-2)));
        d = -V*inv(lambda)*transpose(V)*g;
        
        # Backtracking linesearch
        alpha = 1;
        if linesearch
            mu = 10^-2.0;
            (newf,newg,newH) = obj(x+alpha*d);
            while newf > f + (alpha*mu)*(dot(g,d))
                (newf, newg, newH) = obj(x + alpha*d);
                alpha = alpha/2;
            end
        end
        
        x = x + alpha * d
        
        its += 1
        fvals = [fvals; f]
        gnormvals = [gnormvals; norm(g,2)]
    end
    
    if verbose
        print("Done!\n")
        @printf "Optimal value: %f\n" f
        print("Location: \n")
        print(x)
        print("\n")
        @printf "Iterations: %d\n" its
        print("\n\n")
    end
    
    return (x,f,norm(g,2),its)
end

newtonMethod (generic function with 1 method)

In [26]:
# Loop to test different \beta values
for b = 2:8
    function obj(x)
        f = norm(x,2)^b
        g = x*b*norm(x,2)^(b-2)
        H = b*((b-2)*norm(x,2)^(b-4)*x*x' + norm(x,2)^(b-2)*ones(size(x))*ones(size(x))')
        return (f,g,H)
    end
    
    # Loop to try different x_0 points
    for j = 1:10
        x0 = 10*randn(5,);
        (x,f,normg,its) = newtonMethod(obj,x0,linesearch=false,maxIts=10);
        @printf "Iteration: %2d | Beta: %2.2f | Starting norm: %4.4f | x-norm: %4.4f\n\n" j b norm(x0,2) f
    end
end

Iteration: 

LoadError: LoadError: MethodError: `isless` has no method matching isless(::Float64, ::Array{Complex{Float64},1})
Closest candidates are:
  isless(::Float64, !Matched::Float64)
  isless(::AbstractFloat, !Matched::AbstractFloat)
  isless(::Real, !Matched::AbstractFloat)
  ...
while loading In[26], in expression starting on line 2

 1 | Beta: 2.00 | Starting norm: 7.9014 | x-norm: 4897401552310205102600083262597193231499264.0000

Iteration:  2 | Beta: 2.00 | Starting norm: 16.7299 | x-norm: 66805371768545066476874187043981896403910656.0000

Iteration:  3 | Beta: 2.00 | Starting norm: 14.3742 | x-norm: 29071007305920772919949555188787904138706944.0000

Iteration:  4 | Beta: 2.00 | Starting norm: 20.9539 | x-norm: 71099454311449104169179134318862898661687296.0000

Iteration:  5 | Beta: 2.00 | Starting norm: 17.0967 | x-norm: 64777667636550562627748753782536057787514880.0000

Iteration:  6 | Beta: 2.00 | Starting norm: 31.6719 | x-norm: 112065468483329341325627254031276325442945024.0000

Iteration:  7 | Beta: 2.00 | Starting norm: 22.2877 | x-norm: 108206129052693285721200447297749332879474688.0000

Iteration:  8 | Beta: 2.00 | Starting norm: 16.1134 | x-norm: 35965692591921949595633844334669096625897472.0000

Iteration:  9 | Beta: 2.00 | Starting norm: 21.2944 | x-norm: 104368148589495952778590222260076624882958336

c) The Armijo linesearch is implemented in the following code box. I randomly sampled points in a radius-10 ball from the origin and tried all values of $\beta$ from 2 to 8. They all reduced the norm of the point! This is a significant improvement from part b.

In [8]:
# Loop to test different \beta values
for b = 2:8
    # Loop to try different x_0 points
    for j = 1:10
            x0 = 10*randn(5,);

            function obj(x)
                f = norm(x,2)^b
                g = x*b*norm(x,2)^(b-2)
                H = b*((b-2)*norm(x,2)^(b-4)*x*x' + norm(x,2)^(b-2)*ones(size(x))*ones(size(x))')
                return (f,g,H)
            end

        (x,f,normg,its) = newtonMethod(obj,x0,linesearch=true)

            # Iterate
            x = x + alpha * d;
        end
        
        @printf "Iteration: %2d | Beta: %2.2f | Starting norm: %4.4f | x-norm: %4.4f\n\n" j b norm(x0,2) norm(x,2)
    end
end

Iteration:  1 | Beta: 2.00 | Starting norm: 17.9916 | x-norm: 9.3288

Iteration:  2 | Beta: 2.00 | Starting norm: 30.1060 | x-norm: 10.3129

Iteration:  3 | Beta: 2.00 | Starting norm: 15.5384 | x-norm: 9.5009

Iteration:  4 | Beta: 2.00 | Starting norm: 21.6784 | x-norm: 4.8251

Iteration:  5 | Beta: 2.00 | Starting norm: 23.5745 | x-norm: 10.0854

Iteration:  6 | Beta: 2.00 | Starting norm: 15.1349 | x-norm: 4.4240

Iteration:  7 | Beta: 2.00 | Starting norm: 14.6318 | x-norm: 3.7839

Iteration:  8 | Beta: 2.00 | Starting norm: 23.9553 | x-norm: 6.5890

Iteration:  9 | Beta: 2.00 | Starting norm: 14.3567 | x-norm: 1.8709

Iteration: 10 | Beta: 2.00 | Starting norm: 20.2260 | x-norm: 0.0770

Iteration:  1 | Beta: 3.00 | Starting norm: 13.0066 | x-norm: 4.3410

Iteration:  2 | Beta: 3.00 | Starting norm: 8.2552 | x-norm: 2.3606

Iteration:  3 | Beta: 3.00 | Starting norm: 16.7944 | x-norm: 7.1489

Iteration:  4 | Beta: 3.00 | Starting norm: 15.8737 | x-norm: 3.3764

Iteration:  5 | Bet

# Problem 3
Suppose $D^0$ is symmetric and that $D^k$ is updated according to the formula 

$$D^{k+1} = D^k + \frac{y^k y^{k'}}{q^{k'} y^{k}}$$

where $y^k = p^k - D^k q^k$. Show that we have

$$D^{k+1} q^i = p^i$$ 

for all $k$ and $i \leq k$. 

Conclude that for a positive definite quadratic problem, after $n$ steps for which $n$ linearly independent increments $q^0, ..., q^{n-1}$ are obtained, $D^n$ is equal to the inverse Hessian of the cost function.

## Solution
Don't have this one yet. An update is coming!

# Problem 4
Add BFGS quasi-Newton option to the previous Newton method and compare it with the regular Newton on Toms566 problems.

## Solution
The code is below.

## BFGS

In [74]:
function bfgsMethod(obj, x0; maxIts = 500, optTol = 1.0e-8, verbose=false)
    # Initialize parameters.
    its = 0
    x = x0
    fvals = []
    gnormvals = []
    (f,g,_) = obj(x)
    g0 = g
    B = eye(length(g))
    
    while (its < maxIts && norm(g,2) > optTol && norm(g,2) > (1.0e-4*norm(g0,2)))
        (f,g,_) = obj(x)
        
        # Modify B if not positive definite.
        #E = eigfact(B);
        #V = E[:vectors];
        #lambda = diagm(max(E[:values],1e-2));
        #d = -V*inv(lambda)*transpose(V)*g;
        
        d = B \ (-g)
        
        # Backtracking linesearch.
        alpha = 1.0;
        mu = 10^-1.0;
        (newf,newg,_) = obj(x+alpha*d);
        while newf > f + (alpha*mu)*(dot(g,d))
            (newf,newg,_) = obj(x + alpha*d);
            alpha = alpha/2;
        end
        
        # Update x.
        x = x + alpha * d
        
        # Perform the BFGS update.
        s = alpha*d
        y = newg - g
        B = B + (y*y') / dot(y,s) - ((B*s)*(s'*B)) / dot(s,B*s)
        
        # Increment and store data.
        its += 1
        fvals = [fvals; f]
        gnormvals = [gnormvals; norm(g,2)]
    end
    
    # Maybe pretty print things.
    if verbose
        print("Done!\n")
        @printf "Optimal value: %f\n" f
        print("Location: \n")
        print(x)
        print("\n")
        @printf "Iterations: %d\n" its
        print("\n\n")
    end
    
    return (x,f,norm(g,2),its)
end

bfgsMethod (generic function with 1 method)

In [90]:
print("Running Dmitry's Newton method on all problems!\n\n")
@printf "%5s %30s %20s %20s %20s \n" "i" "Problem name" "f(x)" "|grad(f(x))|" "its"

for i = 1:18
    p = Problem(i)
    
    function obj(x)
        return (p.obj(x), p.grd(x), p.hes(x))
    end
    
    (x,f,gnorm,its) = newtonMethod(obj,p.x0)
    @printf "%5d %30s %20f %20f %20d \n" i p.name f gnorm its
end

Running Dmitry's Newton method on all problems!

    i                   Problem name                 f(x)         |grad(f(x))|                  its 
    1                Hellical valley             0.000010             0.110821                   35 
    2                    Bigg's EXP6             0.255601             0.138184                  500 
    3                       Gaussian             0.000000             0.000000                    3 
    4                         Powell             0.000101             1.999961                   28 
    5                      Box 3-dim             0.000165             0.001989                   19 
    6           Variably dimensioned         31543.569717       1392845.689824                    4 
    7                         Watson             3.527223             5.749285                  500 
    8                      Penalty I          6396.051688          2865.310581                    9 
    9                     Penalty II      

In [91]:
print("Running Dmitry's BFGS method on all problems!\n\n")
@printf "%5s %30s %20s %20s %20s \n" "i" "Problem name" "f(x)" "|grad(f(x))|" "its"

for i = 1:18
    p = Problem(i)
    
    function obj(x)
        return (p.obj(x), p.grd(x), p.hes(x))
    end
    
    (x,f,gnorm,its) = bfgsMethod(obj,p.x0)
    @printf "%5d %30s %20f %20f %20d \n" i p.name f gnorm its
end

 18                      Chebyquad             0.008949             1.018371                  500 
Running Dmitry's BFGS method on all problems!

    i                   Problem name                 f(x)         |grad(f(x))|                  its 
    1                Hellical valley             0.000004             0.046511                   24 
    2                    Bigg's EXP6             0.005656             0.000124                   36 
    3                       Gaussian             0.000000             0.000000                    6 
    4                         Powell             0.000000             1.066644                   59 
    5                      Box 3-dim             0.020431             0.010411                   17 
    6           Variably dimensioned        425274.040251       9907956.334987                    8 
    7                         Watson             0.000131             0.005266                   29 
    8                      Penalty I          